## Pizza Classifier
Determines whether an image contains a pizza<br>
Created for CS50<br>
Dataset: https://www.kaggle.com/datasets/carlosrunner/pizza-not-pizza

In [1]:
import cv2
import cupy as np
from importlib import reload
from os import listdir
from sandbox import model, layers, activations, costs, utils

In [2]:
# Load altered pizza dataset
dataset_path = 'dataset\\altered\\'
datasets = [
    np.array([
        cv2.imread(dataset_path + dir + img).flatten()
        for img in listdir(dataset_path + dir)
    ])
    for dir in ['train\\pizza\\', 'train\\not_pizza\\', 'test\\pizza\\', 'test\\not_pizza\\']
]

# Create training and testing data
train_x = np.concatenate((datasets[0], datasets[1]), axis=0) / 255
train_y = np.concatenate((np.ones((datasets[0].shape[0], 1)), np.zeros((datasets[1].shape[0], 1))), axis=0)

test_x = np.concatenate((datasets[2], datasets[3]), axis=0) / 255
test_y = np.concatenate((np.ones((datasets[2].shape[0], 1)), np.zeros((datasets[3].shape[0], 1))), axis=0)

In [15]:
reload(model)

# Create model
pizza = model.Model(cuda=True)
pizza.add(layers.Dense(units=32, activation=activations.ReLU()))
pizza.add(layers.Dense(units=16, activation=activations.ReLU()))
pizza.add(layers.Dense(units=8, activation=activations.ReLU()))
pizza.add(layers.Dense(units=1, activation=activations.Sigmoid()))
pizza.configure(learning_rate=0.01, epochs=2500, cost_type=costs.BinaryCrossentropy())

# Train model
pizza.train(train_x, train_y, verbose=True)

Cost on epoch 0: 0.69364
Cost on epoch 100: 0.67845
Cost on epoch 200: 0.62467
Cost on epoch 300: 0.60759
Cost on epoch 400: 0.59527
Cost on epoch 500: 0.58711
Cost on epoch 600: 0.57928
Cost on epoch 700: 0.57113
Cost on epoch 800: 0.56565
Cost on epoch 900: 0.55923
Cost on epoch 1000: 0.55553
Cost on epoch 1100: 0.54682
Cost on epoch 1200: 0.54074
Cost on epoch 1300: 0.53218
Cost on epoch 1400: 0.52783
Cost on epoch 1500: 0.51926
Cost on epoch 1600: 0.51236
Cost on epoch 1700: 0.50551
Cost on epoch 1800: 0.49818
Cost on epoch 1900: 0.4925
Cost on epoch 2000: 0.48984
Cost on epoch 2100: 0.47819
Cost on epoch 2200: 0.47218
Cost on epoch 2300: 0.46587
Cost on epoch 2400: 0.45949
Cost on epoch 2500: 0.45947


In [16]:
# Print model summary
pizza.summary()

# Assess model accuracy
pred_train = pizza.predict(train_x, prediction_type=utils.Predictions.binary_classification) # Get model accuracy on training data
print('\nTraining Accuracy: ' + str(np.round(np.sum((pred_train == train_y)/train_x.shape[0]), decimals=5)))
pred_test = pizza.predict(test_x, prediction_type=utils.Predictions.binary_classification) # Get model accuracy on testing data
print('Testing Accuracy: ' + str(np.round(np.sum((pred_test == test_y)/test_x.shape[0]), decimals=5)))

+------------+------------+
| Layer type | Parameters |
+------------+------------+
|   Dense    |   393248   |
|   Dense    |    528     |
|   Dense    |    136     |
|   Dense    |     9      |
+------------+------------+
Total parameters: 393921

Training Accuracy: 0.80596
Testing Accuracy: 0.70888
